# Projet info classes

In [1]:
# imports
import numpy as np
from uuid import uuid4 # pour identifiants uniques générés automatiquement
import pygraphviz as pgv
from PIL import Image
import csv
import pandas as pd  

In [2]:
import numpy as np

In [3]:
class Node:
    def __init__(self, lat : float, long : float):
        self.id = uuid4()
        self.lat = lat
        self.long = long
        self.children = []  # dico où les cles sont les identifiants, les valeurs [id,  et les valeurs le poids des aretes
    
    
    def new_child(self, node):  # crée un fils 
        # distance au père est la distance euclidienne
        self.children.append(node)
    
    def __eq__(self, node):
        if self.id == node.id:
            return True
        else:
            return False
    
def dist(node1, node2):
        return np.sqrt((node1.lat - node2.lat)**2 +(node1.long - node2.long)**2)
        

In [4]:
class Client(Node): 
    def __init__(self, lat : float, long : float, size = None): # pour le moment 1 colis par client!!!
        self.id = uuid4()
        self.lat = lat
        self.long = long
        self.taille_colis = size
        self.children = []
        self.str = "client"
    
def make_client(lat : float, long : float, size: int):
    new_client = Client(lat, long, size)
    return new_client
    
    
class Colis(Node):
    def __init__(self, size, entrepot: Node, destination):
        self.id = uuid4()
        self.size = size
        self.entrepot = entrepot
        self.client = make_client(destination[0], destination[1], size)
        self.children = []
        

class Garage(Node):
    def __init__(self, lat: float, long: float, nb_camions: int, nb_legers: int):
        self.id = uuid4()
        self.lat = lat
        self.long = long
        self.nb_camions = nb_camions
        self.nb_legers = nb_legers
        self.children = []
        self.str = "garage"

class Entrepot(Node):
    def __init__(self, lat : float, long : float, max_camions : int, max_legers : int, capacite : int):
        self.id = uuid4() # retrouver comment faire des identifiants uniques (avec un itérable?)
        self.lat = lat
        self.long = long
        self.children = []
        self.max_camions = max_camions
        self.max_legers = max_legers
        self.capacite = capacite
        self.str = "entrepot"

class Route: # j'aurais tendance à mettre ces infos (enfin etroite uniquement) dans le dico directement
    def __init__(self, vitesse, etroite : bool, dist):
        self.vitesse = vitesse # vitesse max autorisée en km/h
        self.etroite = etroite
        self.dist = dist

In [5]:
class Vehicule:
    def __init__(self, capacite, pollution, dist_max, temps, route_etroite : bool):
        self.capacite = capacite
        self.pollution = pollution
        self.dist_max = dist_max
        self.route_etroite = route_etroite
        self.temps = temps # temps du camioneur 
class Camion(Vehicule):
    def __init__(self, capacite, pollution, dist_max, route_etroite = False):
        self.capacite = capacite
        self.pollution = pollution
        self.dist_max = dist_max
        self.route_etroite = route_etroite
        
class Leger(Vehicule):
    def __init__(self, capacite, pollution = 0, dist_max = 10, route_etroite = True):
        self.capacite = capacite
        self.pollution = pollution
        self.dist_max = dist_max
        self.route_etroite = route_etroite    

### Construction du graphe

+ Tous les noeuds peuvent etre obtenus en descendant depuis la racine (garage) vers ses enfants successifs
+ Il y a 3 types de noeuds (générations): 
    * garage 
    * entrepots 
    * points relais
    
On fait les hypothèses suivantes:
+ Il y a un seul garage depuis lequel partent tous les véhicules
+ Les véhicules se répartissent selon les $K$ entrepots en respectant le nombre max de véhicules de ces derniers
+ Pour un entrepot donné, les véhicules ne desservent que les clients dont les colis sont dans cet entrepot. Ainsi, si on oublie la racine, on a un graphe non connexe composé de plusieurs sous-graphes correspondant aux ensembles (entrepot $k$ - clients de l'entrepot $k$). Le pb revient donc à optimiser chacun de ces sous-graphes, indépendamment les uns des autres si la flotte de véhicules est importante (supérieure ou égale à la somme des véhicules max des entrepots).
+ Le véhicule prend tout son stock pour la journée et ne repasse pas par l'entrepot (arete orientée). Il peut aller directement de l'entrepot vers chacun des clients.
+ Le véhicule peut aller d'un client à l'autre (aretes non orientées)

In [6]:
class Graph:
    def __init__(self, garage, entrepots: [Entrepot], points_relais: [Entrepot], colis: [Colis]):
        self.garage = garage #la racine 
        self.entrepots = entrepots # liste des entrepots (fixée)
        self.points_relais = points_relais
        self.colis = colis # liste des colis à livrer le jour n

    def make_graph(self):
        # self.graph_list.append(self.garage)
        for e in self.entrepots:
            self.garage.new_child(e) # arete orientée du garage vers l'entrepot
            colis_e = [] # liste des paquets qui partent de e
            for p in self.colis:
                if p.entrepot == e:
                    colis_e.append(p)
            for p in colis_e:
                e.new_child(p.client) # arete orientée de l'entrepot vers le client
                for pp in colis_e:
                    if pp.id != p.id:
                        p.client.new_child(pp.client) # NB: pour le moment on "perd" le paquet dans la construction du graphe
        
        for r in self.points_relais:
            for p in self.colis:
                if dist(r, p) < 5: # périmètre de 5 km
                    r.new_child(p.client)
                    p.client.new_child(r)  
    
    
    def generate_csv(self):
        numero = 1
        file_names = [] 
        for e in self.entrepots:
            csv_entrepot(e, numero)
            name = "entrepot_"+str(numero)+".csv"
            file_names.append(name)
            numero += 1
        return file_names

def csv_entrepot(e, numero: int):
    # L est une liste de listes
    L = []
    # 1èr élément de la liste correspond aux données de l'entrepot
    l = [e.id, e.capacite, e.max_camions, 0]
    tree_nodes = [e]+e.children
    for n in tree_nodes:
        l.append(dist(e, n))
    L.append(l)
    # les autres éléments de L correspondent aux données des clients de l'entrepot
    for client in e.children:
        l = [client.id, e.capacite, e.max_camions, client.taille_colis]
        for n in tree_nodes:
            l.append(dist(client, n)) # convention: route du noeud client vers l'autre noeud
        L.append(l)
    names = ["Identifiant", "Capacite", "Vehicules_max", "Demande", "entrepot"]
    tree_nodes.pop(0)
    for i in range(len(e.children)):
        ch = "client "+ str(i+1)
        names.append(ch)
    df = pd.DataFrame(L, columns = names)
    name = "entrepot_"+str(numero)+".csv"
    csv = df.to_csv(name)
    return csv

In [7]:
def trace_graph(graph):
    G = pgv.AGraph(directed = True)
    root = graph.garage
    #G.add_node(root)
    file = [root]
    while len(file) >0:
        if isinstance(file[0], Garage):
            G.add_node(file[0], color = "black")
        if isinstance(file[0], Entrepot):
            G.add_node(file[0], color = "blue")
        if isinstance(file[0], Client):
            G.add_node(file[0], color = "pink")
        for c in file[0].children:
            if len(c.children)>0 and (file[0] in c.children):
                G.add_edge(file[0], c, color = "orange", label = str(round(dist(file[0], c), 1)))
                # G.add_edge(c, file[0], color = "orange")

            if len(c.children) > 0 and (file[0] in c.children) == False:
                G.add_edge(file[0], c, color = "blue", label = str(round(dist(file[0], c), 1)))
                file.append(c)
            if len(c.children) == 0:
                G.add_edge(file[0], c, color = "blue", label = str(round(dist(file[0], c), 1)))
                file.append(c)   
        file.pop(0)
    
    G.layout(prog='dot')
    G.draw('file.png') 
    a = Image.open('file.png')
    a.show()

In [12]:
g = Garage (150, 50, 40, 60)
e1 = Entrepot (100, 100, 10, 15, 5000)
e2 = Entrepot (200, 100, 20, 10, 4000)
e3 = Entrepot (100, 300, 15, 15, 4000)
p1 = Colis (20, e1, [104, 120])
p2 = Colis (30, e1, [104, 135])
p3 = Colis (15, e2, [100, 135])
p4 = Colis (23, e2, [103, 123])
p5 = Colis (12, e1, [112, 122])
p6 = Colis (13, e1, [107, 108])
p7 = Colis (25, e3, [133, 123])
entrepots = [e1, e2, e3]
points_relais = []
paquets = [p1, p2, p3, p4, p5, p6, p7]
G = Graph(g, entrepots, points_relais, paquets)
G.make_graph()
G.garage.children[0].children
trace_graph(G)
file_names = G.generate_csv()
file_names

['entrepot_1.csv', 'entrepot_2.csv', 'entrepot_3.csv']

Coucou Sophie c'est Emma je t'explique ce qu'il me faut hihi


Mon algo s'applique à un entrepot en particulier, il faudrait donc que je puisse le faire tourner n fois (ou n est le nombre d'entrepots). Pour ça j'aurais besoin d'un fichier d'entrée csv qui ressemble à ça :

In [9]:
import pandas as pd
exemple = pd.read_csv('exemple.txt', sep='\t')

In [26]:
print(exemple.head())
exemple['0'][1]

   CUST ID  CAPACITY  VEHICLE_NUMBER  DEMAND     0     1     2       3
0        0       200              25       0     0  dist  dist    dist
1        1       200              25      10  dist     0  dist  dist  
2        2       200              25      30   NaN   NaN   NaN     NaN
3        3       200              25      10   NaN   NaN   NaN     NaN
4        4       200              25      10   NaN   NaN   NaN     NaN


'dist'

Alors je t'explique les notations:
- le cust_id c'est le numéro du client qu'on renumérote de 0 à n et 0 c'est l'entrepot à partir duquel partent tous les colis qui lui sont rattachés.

- la capacity = c'est la même pour tous les camions !

- le vehicle number : c'est le nombre maximum de camions qui partent de l'entrepot

- et après le 0, 1, 2, 3 ça va correspondre à la matrice des distances entre les différents clients (et avec l'entrepot bien sur)!


Est ce que tu penses que c'est bof comme truc? J'aimerais faire la chose la plus simple en fait